In [1]:
from helper import pull_records
import pandas as pd
import metadata_parser
import json
import pickle


In [2]:
def get_meta(row):
    """
    get meta data from a url
    """
    meta = []
    
    try:
        print(f"trying row {row['title']}")
        page = metadata_parser.MetadataParser(url=row["link"])
        meta.append(page.metadata["og"])
    except Exception as e:
        print(f"oopsies on row {row['title']}")
    else:
        print(f"success row {row['title']}")

    return meta


def scrape_metadata(records, export=False):
    """
    get meta data from a list of records
    """
    #only try to get metadata for records we dont already have
    try:
        with open("clean_meta.json", "r") as file:
            clean_meta = json.load(file)
    except:
        clean_meta = []

    try:
        with open("old_urls.pkl", "rb") as file:
            old_urls = pickle.load(file)
    except:
        old_urls = []

    print('before: ', len(records))
    records = records[~records["link"].isin(old_urls)]
    bad_records = ['Why police so rarely get charged for killing unarmed black men like Stephon Clark', "America’s Most Diverse City Is Still Scarred by Its Past", "No game days. No bars. The pandemic is forcing some men to realize they need deeper friendships."]
    records = records[~records["title"].isin(bad_records)] 

    urls = records['link'].tolist()

    print('after: ', len(records))
    i = 0
    clean_meta_dict = []
    while i <= len(records):

        print(f"getting meta for {i} to {i + 100}")
        records_meta = records[i:i + 100]      
        meta = records_meta.apply(get_meta, axis=1)
        print(len(meta))
        clean_meta_dict = clean_meta_dict + [r[0] for r in meta.tolist() if len(r) > 0 and len(r[0]) > 0]
        i += 100
    
    # combine clean_meta_dict and clean_meta
    clean_meta_dict = clean_meta_dict + clean_meta

    # save old urls
    old_urls = old_urls + urls

    with open("old_urls.pkl", "wb") as file:
        pickle.dump(old_urls, file)


    #clean_meta
    if export:
        with open("clean_meta.json", "w") as fp:
            json.dump(clean_meta_dict, fp)


In [4]:
with open("clean_meta.json", "r") as file:
    clean_meta = json.load(file)

with open("old_urls.pkl", "rb") as file:
    old_urls = pickle.load(file)


In [5]:
len(clean_meta)


238

In [16]:
clean_meta[0]['title']

'Opinion | The Unmet Promise of Equality (Published 2018)'

In [19]:
[m for m in clean_meta if "Delhi" in m.get('title','9')]

[{'url': 'https://www.nytimes.com/interactive/2020/12/17/world/asia/india-pollution-inequality.html',
  'type': 'article',
  'title': 'Who Gets to Breathe Clean Air in New Delhi? (Published 2020)',
  'image': 'https://static01.nyt.com/images/2020/12/28/world/delhi-air-promo/delhi-air-promo-facebookJumbo.jpg?year=2020&h=550&w=1050&s=7d5ac8eb518014e40c586399d167d29fcd789b118bc215b3779bc7e1b49e569e&k=ZQJBKqZ0VN',
  'image:alt': '',
  'description': 'We measured how much air pollution two children, Monu and Aamya, breathed for a day to see how much inequality makes a difference.'}]

In [7]:
len(old_urls)

358

In [8]:
records = pull_records()
scrape_metadata(records,export = True)

before:  361
after:  0
getting meta for 0 to 100
trying row nan
oopsies on row nan
0
